## Analisis de Chicago (Crimines - Escuelas - Localidades)

In [ ]:
# instalar Libretias
# !pip install -q pandas==1.1.5
# !pip install ipython-sql

# Cargar Librerias 
import csv, sqlite3
import pandas

# conectar BBDD
%load_ext sql
%sql sqlite:///BDExport.db

con = sqlite3.connect("BDExport.db")
cur = con.cursor()

In [ ]:
# cargar csv
df = pandas.read_csv("ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

In [ ]:
# muestra tablas
%sql SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
# muestra columnas
# %sql PRAGMA table_info(CENSUS_DATA);
# %sql PRAGMA table_info(CHICAGO_CRIME_DATA);
# %sql PRAGMA table_info(CHICAGO_PUBLIC_SCHOOLS_DATA);

# renombra columna
# %sql ALTER TABLE CHICAGO_PUBLIC_SCHOOLS_DATA RENAME 'Elementary, Middle, or High School' TO 'ELEMENTARY_MIDDLE_OR_HIGH_SCHOOL';%sql ALTER TABLE CHICAGO_PUBLIC_SCHOOLS_DATA RENAME 'Elementary, Middle, or High School' TO 'Elementary_Middle_or_High_School';

* Problema 1: Encuentra el número total de delitos registrados en la tabla CRIMEN 

In [120]:
%sql select count('CASE_NUMBER') as 'Total delitos:' from CHICAGO_CRIME_DATA;

 * sqlite:///BDCrimenes.db
Done.


Total delitos:
533


* Problema 2: enumere las áreas comunitarias con un ingreso per cápita inferior a 11000.

In [121]:
%sql select COMMUNITY_AREA_NAME as 'Areas Comunitarias < 11000:' from CENSUS_DATA where PER_CAPITA_INCOME < 11000;

 * sqlite:///BDCrimenes.db
Done.


Areas Comunitarias < 11000:
West Garfield Park
South Lawndale
Fuller Park
Riverdale


* Problema 3: Enumere todos los números de casos de delitos que involucran a menores (los niños no se consideran menores a los efectos del análisis del delito ) 

In [122]:
%sql select CASE_NUMBER as 'Casos de Delitos que involucran Niños:' from CHICAGO_CRIME_DATA where PRIMARY_TYPE ='OFFENSE INVOLVING CHILDREN';

 * sqlite:///BDCrimenes.db
Done.


Casos de Delitos que involucran Niños:
HN567387
HR391350
HM768251
HT394616


* Problema 4: Enumere todos los delitos de secuestro que involucran a un niños

In [123]:
%sql select CASE_NUMBER as 'Casos de Delitos de secuestros que involucran Niños:' from CHICAGO_CRIME_DATA where PRIMARY_TYPE ='KIDNAPPING';

 * sqlite:///BDCrimenes.db
Done.


Casos de Delitos de secuestros que involucran Niños:
HN144152


* Problema 5: ¿Qué tipo de delitos se registraron en las escuelas? 

In [124]:
%sql select PRIMARY_TYPE as 'Tipos de Delitos en Escuelas:' from CHICAGO_CRIME_DATA where LOCATION_DESCRIPTION like 'SCHOOL,%';

 * sqlite:///BDCrimenes.db
Done.


Tipos de Delitos en Escuelas:
BATTERY
BATTERY
BATTERY
BATTERY
BATTERY
CRIMINAL DAMAGE
NARCOTICS
NARCOTICS
ASSAULT
CRIMINAL TRESPASS


* Problema 6: Indique el puntaje promedio de seguridad para todos los tipos de escuelas

In [125]:
%sql select ELEMENTARY_MIDDLE_OR_HIGH_SCHOOL as 'Tipo de Escuela:', avg(SAFETY_SCORE) as 'Promedio:' from CHICAGO_PUBLIC_SCHOOLS_DATA group by ELEMENTARY_MIDDLE_OR_HIGH_SCHOOL order by ELEMENTARY_MIDDLE_OR_HIGH_SCHOOL;

 * sqlite:///BDCrimenes.db
Done.


Tipo de Escuela:,Promedio:


* Problema 7: Enumere 5 áreas comunitarias con el porcentaje más alto de hogares por debajo de la línea de pobreza

In [126]:
%sql select COMMUNITY_AREA_NAME as 'Area Comunitaria:', PERCENT_HOUSEHOLDS_BELOW_POVERTY as 'Porcentaje:' from CENSUS_DATA order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5;

 * sqlite:///BDCrimenes.db
Done.


Area Comunitaria:,Porcentaje:
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


* Problema 8: ¿Qué área comunitaria (número) es más propensa a la delincuencia? 

In [127]:
%sql select COMMUNITY_AREA_NUMBER as 'Area Comunitaria:'from CHICAGO_CRIME_DATA group by COMMUNITY_AREA_NUMBER order by count(COMMUNITY_AREA_NUMBER) desc limit 1;

 * sqlite:///BDCrimenes.db
Done.


Area Comunitaria:
25.0


* Problema 9: utilice una subconsulta para encontrar el nombre del área comunitaria con el índice de dificultad más alto

In [128]:
%sql select COMMUNITY_AREA_NAME as 'Area Comunitaria:', HARDSHIP_INDEX as 'Indice de dificultad:' from CENSUS_DATA where HARDSHIP_INDEX = (select MAX(HARDSHIP_INDEX) from CENSUS_DATA);

 * sqlite:///BDCrimenes.db
Done.


Area Comunitaria:,Indice de dificultad:
Riverdale,98.0


* Problema 10: ¿Usar una subconsulta para determinar el nombre del área comunitaria con la mayor cantidad de delitos? 

In [129]:
%sql select COMMUNITY_AREA_NAME as 'Area Comunitaria:'from CENSUS_DATA where COMMUNITY_AREA_NUMBER = (select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA group by COMMUNITY_AREA_NUMBER order by count(COMMUNITY_AREA_NUMBER) desc limit 1);

 * sqlite:///BDCrimenes.db
Done.


Area Comunitaria:
Austin
